In [8]:
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import glob
import time
import pandas as pd
from vzorci_in_pomozne_definicije import *

    
# Tole dvoje je treba, da se Chrome sproti ne odpira
chrome_options = Options()
chrome_options.add_argument("--headless")
service_obj = Service("chromedriver.exe")

podatki_s_spletnih_strani_se_niso_lokalno_shranjeni = False

Najprej s strani https://www.mtgstocks.com poberemo podatke o setih, ki jih želimo analizirati. Pri tem nekatere manj zanimive izpustimo.

In [7]:
# Spodnja celica se požene samo v primeru, da podatki še niso lokalno shranjeni (oz. če spremenimo vrednost bool spremenljivke)
if podatki_s_spletnih_strani_se_niso_lokalno_shranjeni:
    driver = webdriver.Chrome(service = service_obj, options=chrome_options)
    PRVI_SET = 1
    ZADNJI_SET = 1300

    for st_strani in range(PRVI_SET, ZADNJI_SET + 1):
        url = (
            f'https://www.mtgstocks.com/sets/{st_strani}' 
        )
        print(f"\n Zajemam {url}\n")
        driver.get(url)
        time.sleep(1)
        
        
        vsebina = driver.page_source
        naslov_strani = driver.title
        
        stevilo_podstrani_s_kartami = vsebina.count("pagination-page page-item")
        st_besed_Market_na_strani = vsebina.count(">Market<")
        
        print("Naslov strani:", naslov_strani)
        
        if naslov_strani in ["File not found - MTGStocks"]:
            print(f'\n Stran št. {st_strani} ni dosegljiva.\n')
        else:
            print(f'\n Stran št. {st_strani} JE dosegljiva.\n')
            if "Art Series:" in naslov_strani or "From the Vault:" in naslov_strani or st_besed_Market_na_strani == 0:
                continue
            else:
                with open(pridobi_ustrezno_ime_lokalne_datoteke(st_strani), 'w', encoding = "utf-8") as dat:
                    driver.find_element(by=By.XPATH, value = "//*[text() = 'Market']//..").click()
                    for podstran in range(1, stevilo_podstrani_s_kartami + 1):
                        driver.find_element(by=By.XPATH, value = f"//*[contains(@class, 'pagination-page page-item')][{podstran}]//a").click()
                        time.sleep(1)
                        print(f"Pobrana {podstran}. podstran")
                        dat.write(driver.page_source)
    

Nato preberemo lokalno shranjene podatke o setih, ter izluščimo zgolj osnovne podatke o kartah (id, ime, set, redkost, povprečna cena, povprečna cena foila), ki jih zapišemo v csv in json datoteki. 

In [ ]:
POT_DO_LOKALNIH_PODATKOV_O_SETIH = os.path.join('Podatki','Lokalni_podatki_o_setih')
karte = []

# Najprej poradiramo csv in json datoteki
os.remove(FILENAME_KARTE_CSV)
os.remove(FILENAME_KARTE_JSON)

for filename in glob.glob(os.path.join(POT_DO_LOKALNIH_PODATKOV_O_SETIH, '*.html')):
    with open(filename, 'r', encoding = "latin1") as f:
        vsebina = f.read()
        for blok in vzorec_bloka.finditer(vsebina):
            karte.append(izlusci_podatke_o_kartah_iz_bloka(blok.group(0)))

with open(FILENAME_KARTE_JSON, "a", encoding = "utf-8") as dat:
    json.dump(karte, dat, indent=4, ensure_ascii=False)
    
with open(FILENAME_KARTE_CSV, "a", encoding = "utf-8") as dat:
    writer = csv.DictWriter(dat, [
        "id_karte",
        "ime",
        "set",
        "redkost",
        "povprecna_cena",
        "povprecna_cena_foil",
    ])
    writer.writeheader()
    writer.writerows(karte)

print("Prebrali smo lokalno shanjene podatke iz HMTL datotek in jih zapisali v 'karte.csv' in 'karte.json'.")

S pomočjo teh podatkov ustvarimo dataframe. Iz njega izločimo basic lande in tokene. Prav tako izločimo karte, ki nimajo podane niti ene cene.

*Opomba*: Ker se imena nekaterih setov na strani MTGStocks razlikujejo od uradnih, jih popravimo s pomočjo slovarja.

In [ ]:
podatki_o_kartah = pd.read_csv(FILENAME_KARTE_CSV, encoding="latin1")

seznam_basic_landov = ["Forest", "Mountain", "Swamp", "Plains", "Island", "Wastes"]
regex_izraz_za_basic_lande = r'Forest \(\d|Mountain \(\d|Swamp \(\d|Plains \(\d|Island \(\d|Wastes \(\d'
podatki_o_kartah = podatki_o_kartah[~(podatki_o_kartah.redkost.isin(["Token", "Land"])) & ~(podatki_o_kartah.ime.str.match(regex_izraz_za_basic_lande)) & ~(podatki_o_kartah.ime.isin(seznam_basic_landov))]
podatki_o_kartah = podatki_o_kartah[~(podatki_o_kartah.povprecna_cena.isnull() & podatki_o_kartah.povprecna_cena_foil.isnull())]
podatki_o_kartah = podatki_o_kartah.replace(slovar_uradnih_imenovanj_setov)
okrajsani_indeksi = podatki_o_kartah.id_karte # To so indeksi, s katerimi bomo pognali ustrezne url-je
raznoliki_seti = podatki_o_kartah["set"].unique()


Podatke o setih najdemo na strani "https://scryfall.com/sets".

In [ ]:
# Spodnja celica se požene samo v primeru, da podatki še niso lokalno shranjeni (oz. če spremenimo vrednost bool spremenljivke)
if podatki_s_spletnih_strani_se_niso_lokalno_shranjeni:
    driver = webdriver.Chrome(service = service_obj, options=chrome_options)
    driver.get(URL_DO_STRANI_SETOV)
    time.sleep(1)
    vsebina_strani_setov = driver.page_source
    driver.close()

    slovarji_posameznih_setov = []

    os.remove(FILENAME_SETI_CSV)
    with open(FILENAME_SETI_CSV, "a", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, [
                "set",
                "polno_ime",
                "st_kart",
                "datum_izida",
            ])
        
        for posamezen_set in raznoliki_seti:
            vzorec_za_podatke_o_setu = re.compile(
                r'   (?P<polno_ime>\w.*?) <small>' + posamezen_set + r'</small>\n</a></td>.*?<td>\n.*?>(?P<st_kart>.*?)</a>\n.*?\n.*?\n.*?>(?P<datum_izida>.*?)</a>',
            )
            print("Shranjen", posamezen_set, end = " ")
            slovarji_posameznih_setov.append(izlusci_podatke_o_setih(vsebina_strani_setov, vzorec_za_podatke_o_setu, posamezen_set, podatki_o_kartah))
            
        writer.writeheader()
        writer.writerows(slovarji_posameznih_setov)
        
        os.remove(FILENAME_SETI_JSON)
        with open(FILENAME_SETI_JSON, "a", encoding = "utf-8") as dat:
            json.dump(slovarji_posameznih_setov, dat, indent=4, ensure_ascii=False)

S pomočjo indeksov kart, ki smo jih dobili na strani MTGStocks, pridobimo podatke za vse karte iz dataframa.

Na tej točki bi omenil, da sem zaradi osebnega interesa pridobil veliko več podatkov, kot bi bilo potrebno v okviru naloge.

In [ ]:
# Spodnja celica se požene samo v primeru, da podatki še niso lokalno shranjeni (oz. če spremenimo vrednost bool spremenljivke)
if podatki_s_spletnih_strani_se_niso_lokalno_shranjeni:
    driver = webdriver.Chrome(service = service_obj, options=chrome_options)
    print("Število vseh indeksov kart, ki jih imamo:", len(okrajsani_indeksi))

    os.remove(FILENAME_KARTE_DODATEK_CSV)
    with open(FILENAME_KARTE_DODATEK_CSV, "a", encoding = "utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, [
            "id_karte",
            "cmc",
            "manacost",
            "barva",
            "povprecje_eu",
            "supertype",
            "cardtype",
            "subtype",
            "oracle_text",
            "reserved_list",
            "all_time_low",
            "all_time_low_datum",
            "all_time_high",
            "all_time_high_datum",
        ])
        
        slovarji_posameznih_kart = []
        stetje_zajetih_kart = 0

        for indeks_printa in okrajsani_indeksi:
            url = f"https://www.mtgstocks.com/prints/{indeks_printa}"
            stetje_zajetih_kart += 1
            print(f"({stetje_zajetih_kart}.) Zajemanje printa karte št. {indeks_printa} s strani {url}")

            try:
                driver.get(url)
                time.sleep(1)
                vsebina = driver.page_source
                glava_slovarja = {"id_karte": indeks_printa}
                manacost_in_barve = izlusci_podatke_manacosta_in_barve(vsebina)
                glavnina = izlusci_podatke_o_specificni_karti_iz_njene_datoteke(vsebina)
                glava_slovarja.update(manacost_in_barve)
                glava_slovarja.update(glavnina)
                print(glava_slovarja, "\n")
                slovarji_posameznih_kart.append(glava_slovarja)
                
            except Exception as e:
                print(f"Prišlo je do napake pri setu {indeks_printa}")
                print(e)
                
        writer.writeheader()
        writer.writerows(slovarji_posameznih_kart)